# SKMULTILEARN

In [24]:
import numpy as np 
import pandas as pd

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import multilabel_confusion_matrix
import sklearn.metrics as skm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.multiclass import OneVsOneClassifier 
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

In [26]:
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN

In [27]:
import feather
import warnings
warnings.filterwarnings('ignore')

As seen in the EDA part the proportion of genres in the dataset is not balenced .We'll choose the model score to be the F1 score.

### Loading the test set in original format

In [252]:
# load data
url1 = "https://raw.githubusercontent.com/PicoRadia/sdataProjet/main/dataset/Spotify_test_dataset.csv"
df = pd.read_csv(url1)#31728 rows x 20 columns
# dropping duplicates
df.drop_duplicates(subset ="id", keep = False, inplace = True)

In [253]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,song_name
0,0.859,0.792,2,-4.007,1,0.1410,0.065000,0.000000,0.3000,0.7660,206.402,audio_features,6tYL1tBaqTcaqXIykzfF7M,spotify:track:6tYL1tBaqTcaqXIykzfF7M,https://api.spotify.com/v1/tracks/6tYL1tBaqTca...,https://api.spotify.com/v1/audio-analysis/6tYL...,276813,4,"All About U (ft. Nate Dogg, Snoop Dogg, Fatal,..."
1,0.829,0.619,1,-7.258,1,0.1980,0.290000,0.000371,0.0975,0.3510,192.070,audio_features,4X5AMbgW6whnAuDX10fhgq,spotify:track:4X5AMbgW6whnAuDX10fhgq,https://api.spotify.com/v1/tracks/4X5AMbgW6whn...,https://api.spotify.com/v1/audio-analysis/4X5A...,148093,4,Revenge
2,0.461,0.834,3,-3.387,0,0.1420,0.261000,0.000000,0.0918,0.5170,129.976,audio_features,2J176Rj3ZTTLKeTirMzQ4M,spotify:track:2J176Rj3ZTTLKeTirMzQ4M,https://api.spotify.com/v1/tracks/2J176Rj3ZTTL...,https://api.spotify.com/v1/audio-analysis/2J17...,193542,4,Changes
3,0.691,0.984,1,-4.828,1,0.0610,0.000924,0.513000,0.5960,0.7560,126.979,audio_features,08oufzbacifu4V5zQcoNMG,spotify:track:08oufzbacifu4V5zQcoNMG,https://api.spotify.com/v1/tracks/08oufzbacifu...,https://api.spotify.com/v1/audio-analysis/08ou...,204091,4,NaN
4,0.685,0.798,6,-4.683,0,0.0342,0.013100,0.000078,0.1110,0.8310,133.994,audio_features,3ZuoDMBWEnM5kuve9Y55tv,spotify:track:3ZuoDMBWEnM5kuve9Y55tv,https://api.spotify.com/v1/tracks/3ZuoDMBWEnM5...,https://api.spotify.com/v1/audio-analysis/3Zuo...,193647,5,Don't Believe Me
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10571,0.768,0.691,11,-8.044,0,0.2060,0.201000,0.000000,0.1820,0.3260,124.987,audio_features,6eRZOb9YH1PDzzKlColIdu,spotify:track:6eRZOb9YH1PDzzKlColIdu,https://api.spotify.com/v1/tracks/6eRZOb9YH1PD...,https://api.spotify.com/v1/audio-analysis/6eRZ...,207513,4,L.A. Vibe
10572,0.844,0.807,1,-8.198,1,0.2330,0.277000,0.000020,0.1550,0.6180,171.968,audio_features,6msTUCqIfVOWe7pzWGzh6F,spotify:track:6msTUCqIfVOWe7pzWGzh6F,https://api.spotify.com/v1/tracks/6msTUCqIfVOW...,https://api.spotify.com/v1/audio-analysis/6msT...,268827,4,Leaving the Past
10573,0.366,0.916,10,-5.609,0,0.4280,0.316000,0.000000,0.0771,0.5240,181.068,audio_features,20QmhsCXRidLu2vI86xVml,spotify:track:20QmhsCXRidLu2vI86xVml,https://api.spotify.com/v1/tracks/20QmhsCXRidL...,https://api.spotify.com/v1/audio-analysis/20Qm...,139690,4,Delorean
10574,0.560,0.993,5,-6.244,1,0.1420,0.000568,0.018500,0.6700,0.1520,139.994,audio_features,6qpfiCAawqyIpwjlx5CJSH,spotify:track:6qpfiCAawqyIpwjlx5CJSH,https://api.spotify.com/v1/tracks/6qpfiCAawqyI...,https://api.spotify.com/v1/audio-analysis/6qpf...,420000,4,NaN


In [29]:
X = pd.read_feather('./fin')
X.head()

,index,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature__1,time_signature__3,time_signature__4,time_signature__5,mode_0,mode_1,mins,song_name_nan
0,1,0.618,0.654,-14.305,0.4220,0.226000,0.864,0.1110,0.1760,127.953,0,0,1,0,1,0,6.068300,0
1,2,0.696,0.896,-5.915,0.1080,0.016400,0.866,0.1030,0.5700,128.001,0,0,1,0,0,1,8.437500,0
2,3,0.769,0.506,-11.658,0.0452,0.002730,0.000,0.0832,0.1020,120.004,0,0,1,0,0,1,3.867433,1
3,5,0.568,0.779,-9.779,0.0323,0.000519,0.929,0.0660,0.0852,128.015,0,0,1,0,0,1,4.100150,0
4,6,0.784,0.852,-2.511,0.1660,0.001630,0.000,0.0781,0.5290,134.046,0,0,1,0,1,0,2.433400,1


In [30]:
a = df.genre
y = pd.get_dummies(a, prefix='t')

## Reading preprocessed test set

In [31]:
test = pd.read_feather('./testfin')
test.head()

,index,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature__1,time_signature__3,time_signature__4,time_signature__5,mode_0,mode_1,mins,song_name_nan
0,0,0.859,0.792,-4.007,0.1410,0.065000,0.000000,0.3000,0.766,206.402,0,0,1,0,0,1,4.613550,1
1,1,0.829,0.619,-7.258,0.1980,0.290000,0.000371,0.0975,0.351,192.070,0,0,1,0,0,1,2.468217,1
2,2,0.461,0.834,-3.387,0.1420,0.261000,0.000000,0.0918,0.517,129.976,0,0,1,0,1,0,3.225700,1
3,3,0.691,0.984,-4.828,0.0610,0.000924,0.513000,0.5960,0.756,126.979,0,0,1,0,0,1,3.401517,0
4,4,0.685,0.798,-4.683,0.0342,0.013100,0.000078,0.1110,0.831,133.994,0,0,0,1,1,0,3.227450,1


In [32]:
test.shape

(10577, 18)

## We won't split the data this time to train the model on the entire training set

In [55]:
classifier = LabelPowerset(
    classifier = RandomForestClassifier(bootstrap=True,n_jobs = -1,class_weight='balanced_subsample'),
    require_dense = [False, True]
)
classifier.fit(X, y)
y_pred = classifier.predict(test)

In [67]:
genre = pd.DataFrame(y_pred.toarray())
genre.columns = y.columns

In [74]:
# extracting song genre from column name of the df genre . Ex : t_Dark Trap=> Dark Trap
for c in genre:
    genre[c].replace(1,c[2:],inplace=True)

In [78]:
genre

,t_Dark Trap,t_Emo,t_Hiphop,t_Pop,t_Rap,t_RnB,t_Trap Metal,t_Underground Rap,t_dnb,t_hardstyle,t_psytrance,t_techhouse,t_techno,t_trance,t_trap
0,0,0,Hiphop,0,0,0,0,0,0,0,0,0,0,0,0
1,Dark Trap,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,RnB,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,techhouse,0,0,0
4,0,0,0,0,0,RnB,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10572,0,0,Hiphop,0,0,0,0,0,0,0,0,0,0,0,0
10573,0,0,Hiphop,0,0,0,0,0,0,0,0,0,0,0,0
10574,0,0,0,0,0,0,0,0,0,0,0,0,0,trance,0
10575,Dark Trap,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [257]:
df.to_csv("predictionfinal.csv")

In [251]:
test

,index,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature__1,time_signature__3,time_signature__4,time_signature__5,mode_0,mode_1,mins,song_name_nan,genre
0,0,0.859,0.792,-4.007,0.1410,0.065000,0.000000,0.3000,0.7660,206.402,0,0,1,0,0,1,4.613550,1,Hiphop
1,1,0.829,0.619,-7.258,0.1980,0.290000,0.000371,0.0975,0.3510,192.070,0,0,1,0,0,1,2.468217,1,Dark Trap
2,2,0.461,0.834,-3.387,0.1420,0.261000,0.000000,0.0918,0.5170,129.976,0,0,1,0,1,0,3.225700,1,RnB
3,3,0.691,0.984,-4.828,0.0610,0.000924,0.513000,0.5960,0.7560,126.979,0,0,1,0,0,1,3.401517,0,techhouse
4,4,0.685,0.798,-4.683,0.0342,0.013100,0.000078,0.1110,0.8310,133.994,0,0,0,1,1,0,3.227450,1,RnB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10572,10572,0.844,0.807,-8.198,0.2330,0.277000,0.000020,0.1550,0.6180,171.968,0,0,1,0,0,1,4.480450,1,Hiphop
10573,10573,0.366,0.916,-5.609,0.4280,0.316000,0.000000,0.0771,0.5240,181.068,0,0,1,0,1,0,2.328167,1,Hiphop
10574,10574,0.560,0.993,-6.244,0.1420,0.000568,0.018500,0.6700,0.1520,139.994,0,0,1,0,0,1,7.000000,0,trance
10575,10575,0.373,0.689,-8.235,0.0468,0.273000,0.021400,0.1290,0.0399,139.909,0,0,1,0,1,0,4.057150,1,Dark Trap
